In [22]:
!conda install -c bioconda pybedtools

^C

CondaError: KeyboardInterrupt



In [23]:
import gffutils
import pybedtools
from pybedtools.featurefuncs import TSS
from gffutils.helpers import asinterval
import bioframe as bf
from gtfparse import read_gtf

In [24]:
tss_df = read_gtf("/project/fudenber_735/collaborations/karissa_2022/old/RNAseq/STAR_Gencode_alignment/tss_annotions_gencode.vM23.primary_assembly.gtf")

INFO:root:Extracted GTF attributes: ['gene_id', 'transcript_id', 'gene_type', 'gene_name', 'transcript_type', 'transcript_name', 'level', 'transcript_support_level', 'mgi_id', 'tag', 'havana_gene', 'havana_transcript', 'protein_id', 'ccdsid', 'ont']


In [25]:
tss_df

,seqname,source,feature,start,end,score,strand,frame,gene_id,transcript_id,...,transcript_name,level,transcript_support_level,mgi_id,tag,havana_gene,havana_transcript,protein_id,ccdsid,ont
0,chr1,HAVANA,transcript,3073252,3073252,NaN,+,0,ENSMUSG00000102693.1,ENSMUST00000193812.1,...,4933401J01Rik-201,2,NA,MGI:1918292,basic,OTTMUSG00000049935.1,OTTMUST00000127109.1,,,
1,chr1,ENSEMBL,transcript,3102015,3102015,NaN,+,0,ENSMUSG00000064842.1,ENSMUST00000082908.1,...,Gm26206-201,3,NA,MGI:5455983,basic,,,,,
2,chr1,HAVANA,transcript,3216345,3216345,NaN,-,0,ENSMUSG00000051951.5,ENSMUST00000162897.1,...,Xkr4-203,2,1,MGI:3528744,,OTTMUSG00000026353.2,OTTMUST00000086625.1,,,
3,chr1,HAVANA,transcript,3215633,3215633,NaN,-,0,ENSMUSG00000051951.5,ENSMUST00000159265.1,...,Xkr4-202,2,1,MGI:3528744,,OTTMUSG00000026353.2,OTTMUST00000086624.1,,,
4,chr1,HAVANA,transcript,3671499,3671499,NaN,-,0,ENSMUSG00000051951.5,ENSMUST00000070533.4,...,Xkr4-201,2,1,MGI:3528744,"basic,appris_principal_1,CCDS",OTTMUSG00000026353.2,OTTMUST00000065166.1,ENSMUSP00000070648.4,CCDS14803.1,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142441,JH584299.1,ENSEMBL,transcript,913084,913084,NaN,-,0,ENSMUSG00000095475.1,ENSMUST00000180208.1,...,AC133095.2-201,3,1,,basic,,,ENSMUSP00000137110.1,,
142442,JH584299.1,ENSEMBL,transcript,921941,921941,NaN,+,0,ENSMUSG00000094855.1,ENSMUST00000178327.1,...,AC133095.1-201,3,5,,basic,,,ENSMUSP00000137276.1,,
142443,JH584303.1,ENSEMBL,transcript,81606,81606,NaN,+,0,ENSMUSG00000095019.1,ENSMUST00000178569.1,...,AC234645.1-201,3,NA,,basic,,,ENSMUSP00000136543.1,,
142444,JH584304.1,ENSEMBL,transcript,59668,59668,NaN,-,0,ENSMUSG00000095041.7,ENSMUST00000179505.7,...,AC149090.1-201,3,1,,basic,,,ENSMUSP00000136028.1,,


In [26]:
# cleaning out less-well defined chromosome numbers
tss_df = tss_df.loc[False==( tss_df['seqname'].str.contains('NT_'))]
tss_df = tss_df.loc[False==( tss_df['seqname'].str.contains('MT'))]

# paste 'chr' to all chromosome names
tss_df["seqname"] = tss_df["seqname"]

# rename column to chrom to match bedframe/bioframe format
tss_df = tss_df.rename(columns= {"seqname" : "chrom"})

# Removing pseudo chromosomes
before = len(tss_df)
tss_df = tss_df.loc[False==( tss_df['chrom'].str.contains('chrGL'))]
tss_df = tss_df.loc[False==( tss_df['chrom'].str.contains('chrJH'))]
tss_df = tss_df.loc[False==( tss_df['chrom'].str.contains('chrX'))]
tss_df = tss_df.loc[False==( tss_df['chrom'].str.contains('chrY'))]
tss_df = tss_df.loc[False==( tss_df['chrom'].str.contains('chrM'))]
tss_df =tss_df.loc[True==tss_df['chrom'].str.contains('chr')]
after = len(tss_df)

In [ ]:
tss_df['chrom'].value_counts()

In [31]:
tss_df[False == (tss_df['chrom'].str.contains("chr"))]

,chrom,source,feature,start,end,score,strand,frame,gene_id,transcript_id,...,transcript_name,level,transcript_support_level,mgi_id,tag,havana_gene,havana_transcript,protein_id,ccdsid,ont


In [32]:
# drop duplicates (same across all columns we are interested in for defining intervals)
ts_intervals = tss_df[['gene_id','chrom', 'start', 'end']].drop_duplicates(['gene_id'])

In [33]:
ts_intervals

,gene_id,chrom,start,end
0,ENSMUSG00000102693.1,chr1,3073252,3073252
1,ENSMUSG00000064842.1,chr1,3102015,3102015
2,ENSMUSG00000051951.5,chr1,3216345,3216345
5,ENSMUSG00000102851.1,chr1,3252756,3252756
6,ENSMUSG00000103377.1,chr1,3368550,3368550
...,...,...,...,...
134932,ENSMUSG00000117740.1,chr19,61232927,61232927
134933,ENSMUSG00000117782.1,chr19,61255966,61255966
134934,ENSMUSG00000117951.1,chr19,61257790,61257790
134935,ENSMUSG00000095993.1,chr19,61297070,61297070
